In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [47]:
!pip install geopandas
!pip install rasterio
!pip install attrdict
!pip install utm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for utm: filename=utm-0.7.0-py3-none-any.whl size=6108 sha256=90d03a024a4a9a5d31cd59f26caafbcdf6d361efe80fe4f32e7a7c767913f378
  Stored in directory: /root/.cache/pip/wheels/a5/b0/12/7ee4fdb0f9fbb4157100bd02390436ed5d58ebfd3c6d6a0886
Successfully built utm


In [1]:
import numpy as np
import gdal
import geopandas as gpd
import pandas as pd
import os
from shapely.geometry import Point, Polygon, MultiPolygon
from pprint import pprint

target_country = 'australia'

base_path = '/content/drive/MyDrive/PyPSA_Africa_images/'
duke_path = os.path.join(base_path, 'duke')
country_path = os.path.join(base_path, target_country)

duke_countries = os.listdir(duke_path)
print(duke_countries)

['sudan', 'south_sudan', 'china', 'rotorua', 'brazil', 'arizona', 'hartford', 'mexico', 'kansas', 'clyde', 'wilmington', 'dunedin', 'gisborne', 'palmertson', 'tauranga']


In [11]:
arizona1 = pd.read_csv(os.path.join(duke_path, 'arizona', 'raw', 'USA_AZ_Tucson_1.csv'), index_col=0)

In [12]:
arizona1.head()

,Label,Object,Type,X,Y,height,width
0,DT,1,Polygon,11427.372913,11150.107695,12100,11846
1,DT,1,Polygon,11429.752872,11167.957387,12100,11846
2,DT,1,Polygon,11447.602564,11165.577428,12100,11846
3,DT,1,Polygon,11445.222605,11150.107695,12100,11846
4,DT,2,Polygon,11714.157967,11330.984575,12100,11846


In [27]:
pprint(gdal.Info(os.path.join(duke_path, 'sudan', 'raw', 'Sudan_Khartoum_1.tif'), format='JSON'))
print('----')
pprint(gdal.Info(os.path.join(duke_path, 'arizona', 'raw', 'USA_AZ_Tucson_1.tif'), format='JSON'))

# print(gdal.Open(os.path.join(duke_path, 'sudan', 'raw', 'Sudan_Khartoum_1.tif')).GetGeoTransform())
# print(gdal.Open(os.path.join(duke_path, 'sudan', 'lab', 'Sudan_Khartoum_1_4326.tif')).GetGeoTransform())

('Driver: GTiff/GeoTIFF\n'
 'Files: '
 '/content/drive/MyDrive/PyPSA_Africa_images/duke/sudan/raw/Sudan_Khartoum_1.tif\n'
 'Size is 1300, 1300\n'
 'Coordinate System is:\n'
 'GEOGCS["WGS 84",\n'
 '    DATUM["WGS_1984",\n'
 '        SPHEROID["WGS 84",6378137,298.257223563,\n'
 '            AUTHORITY["EPSG","7030"]],\n'
 '        AUTHORITY["EPSG","6326"]],\n'
 '    PRIMEM["Greenwich",0],\n'
 '    UNIT["degree",0.0174532925199433],\n'
 '    AUTHORITY["EPSG","4326"]]\n'
 'Origin = (32.489348400012481,15.520831199937557)\n'
 'Pixel Size = (0.000002700000000,-0.000002700000000)\n'
 'Metadata:\n'
 '  AREA_OR_POINT=Area\n'
 'Image Structure Metadata:\n'
 '  INTERLEAVE=PIXEL\n'
 'Corner Coordinates:\n'
 'Upper Left  (  32.4893484,  15.5208312) ( 32d29\'21.65"E, 15d31\'14.99"N)\n'
 'Lower Left  (  32.4893484,  15.5173212) ( 32d29\'21.65"E, 15d31\' 2.36"N)\n'
 'Upper Right (  32.4928584,  15.5208312) ( 32d29\'34.29"E, 15d31\'14.99"N)\n'
 'Lower Right (  32.4928584,  15.5173212) ( 32d29\'34.29"E, 

In [21]:
inpath = os.path.join(duke_path, 'sudan', 'raw', 'Sudan_Khartoum_1.tif')
outpath = os.path.join(duke_path, 'sudan', 'lab', 'Sudan_Khartoum_1_4326.tif')

warp = gdal.Warp(outpath, inpath, dstSRS='EPSG:4326')
warp = None 

In [ ]:
from osgeo import gdal
from attrdict import AttrDict
import utm
from pyproj import CRS
import os

def get_pixel_size(filename):
    '''
    obtains the pixel size in meters of any raster file
    '''

    holdfile = filename.split('/')[-1]

    info = gdal.Info(filename, format='json')
    epsg = int(info['coordinateSystem']['wkt'].rsplit('"EPSG","', 1)[-1].split('"')[0])

    if epsg != 4326:
        dummyfile = os.path.join(duke_path, 'sudan', 'lab', 'epsgdummy.tif')
        warp = gdal.Warp(dummyfile, filename, dstSRS='EPSG:4326')
        warp = None 
        filename = dummyfile
        info = gdal.Info(filename, format='json')

    # print(f'initial epsg: {epsg}')

    info = AttrDict(info)

    center = info.cornerCoordinates.center
    if center[1] > 0:
        south = True
    else:
        south = False
    
    # print('obtained center coords')
    # print(center)

    as_utm = utm.from_latlon(center[1], center[0])
    # print(f'is in utm zone: {as_utm}')
    
    crs = CRS.from_dict({'proj': 'utm', 'zone': as_utm[2], 'south': south})
    # print(f'resulting in epsg {crs.to_authority()}')

    epsg = crs.to_authority()

    utmpath = os.path.join(duke_path, 'sudan', 'lab', 'dummy.tif')

    warp = gdal.Warp(utmpath, filename, dstSRS=epsg[0]+':'+epsg[1])
    warp = None 

    info = AttrDict(gdal.Info(utmpath, format='json'))
    pixel_size = info.geoTransform[1]
    print(f'Pixel size of {holdfile}: {pixel_size} m')



# filename = os.path.join(duke_path, 'arizona', 'raw', 'USA_AZ_Tucson_1.tif')
australia_imgs = os.path.join(base_path, 'maxar', 'texas', 'raw')

for file in os.listdir(australia_imgs)[:10]:
    file = os.path.join(australia_imgs, file)
    get_pixel_size(file)


Pixel size of 105001002A9CF000-visual.tif: 0.3046461274153037 m
Pixel size of 105001002A9CF000-visual (1).tif: 0.3046694633557989 m
Pixel size of 105001002A9CF000-visual (2).tif: 0.3046878849223795 m
